### Use Wisper model to Analyze the Audio wav files and convert them into Text

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multitasking model that can perform multilingual speech recognition, speech translation, and language identification.

#### Conversational Loader 

In [1]:
import os
import time
## load Wisper API 
import json
import pandas as pd 
import whisper
from whisper.audio import SAMPLE_RATE
import torch 
from collections import OrderedDict
audio_path = '../data/Audio/'
print("Defaul sample rate selected :",SAMPLE_RATE)

Defaul sample rate selected : 16000


/home/samartht/anaconda3/envs/pylangchain/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [2]:
excel_path = "/mnt/e/Personal/Samarth/repository/NLP-Basic2Advanced/DMAC_project/data/Data_Mortgage.xlsx"
excel_data = pd.read_excel(excel_path)
excel_data.dropna(subset='Opportunity',inplace=True)
excel_data.head()

,Date,Company / Account,Opportunity,Unnamed: 3,Lead,Assigned,Priority,Status,Task,Ameyo Recording URL,Call Type,CallDurationInSeconds
1,4/4/2023,Mohammed Jaffer,Mohammed Jaffer,Mohammed Jaffer,NaN,Yaseen Syed Ali,Low,Completed,True,https://prypto-api.aswat.co/surveillance/recor...,Outbound,422
2,4/4/2023,G Abbas,G Abbas,G Abbas,NaN,Yaseen Syed Ali,Low,Completed,True,https://prypto-api.aswat.co/surveillance/recor...,Outbound,237
6,4/4/2023,Ahsan Khan,Ahsan Khan,Ahsan Khan,NaN,Yaseen Syed Ali,Low,Completed,True,https://prypto-api.aswat.co/surveillance/recor...,Outbound,74
11,4/5/2023,Fayiqa Iftikhar,Fayiqa Iftikhar,Fayiqa Iftikhar,NaN,Yaseen Syed Ali,Low,Completed,True,https://prypto-api.aswat.co/surveillance/recor...,Outbound,481
13,4/5/2023,Smith Suresh Shetty,Smith Suresh Shetty,Smith Suresh Shetty,NaN,Yaseen Syed Ali,Low,Completed,True,https://prypto-api.aswat.co/surveillance/recor...,Outbound,269


In [4]:
def download_recording(data):
    url_link = data['Ameyo Recording URL']
    account_name = data['Company / Account']
    assigned_name = data['Assigned']
    os.system('wget {} {}'.format(url_link,'audio_{}_{}.mp3'.format(account_name,assigned_name)))
    return 
# excel_data.apply(lambda x: download_recording(x),axis=1)

audio_files = excel_data['Ameyo Recording URL'].to_list()
print(audio_files)

['https://prypto-api.aswat.co/surveillance/recordings/09e85b8d-4acf-4b67-b846-cc5199775df4.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/e6e2bd30-ca04-4f78-9265-afe32e23e2fc.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/3ebff02b-b650-451d-bfb6-ed7c3a6533ea.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/38b72137-5428-45ae-adce-624900b20bea.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/4f39f94d-7dff-440f-a1b0-2a6c75fdc737.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/210bdc1c-da2b-4cc8-8c71-13a659f1b1ff.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/23566839-ddc9-443d-b4e5-7ade236b9146.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/03dac286-7d7e-414c-93d2-6da513d5546f.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/b82d62eb-eaf7-4ce9-9fc2-76df0e217a39.mp3', 'https://prypto-api.aswat.co/surveillance/recordings/a6dde464-20c0-4f7b-8c04-b88b9258e9cd.mp3', 'https://prypto-api.aswat.co/surveillan

In [ ]:
# Loading the Whisper model into Device 
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

whisper_model_name = 'small' # hyperparameter -- Improving this will improve info 
stime = time.time()
wisper_model = whisper.load_model(whisper_model_name,device=device)
# emotion_classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion')
print("Models loaded into memory -- {:.2f}sec".format(time.time()-stime))

In [ ]:
data_path = '../data/Audio'
# audio_files = [fname for fname in os.listdir(data_path) if fname.endswith(('mp3','wav'))]
# print("Total number of audio files found : ",len(audio_files))

### Conversion from Speech to text with traslation and emotion Detection

The below functions load the audio file with Sampling rate of 16000 hz/s. We clip the audio file to 30 seconds to get the language of the conversation. If the language is not EN we translate and save the info in dict file else we simply translate . 

In [ ]:
conversation_dict = OrderedDict()

#### Get Sentiment analysis From VADER

This will be saved as metadata for analyzing the sentiment of the person talking

In [ ]:
# # Sentiment Analyzer 
# import nltk 
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sia=SentimentIntensityAnalyzer()

# def get_sentiment(text_result):
#     emotion = None 
#     if sia.polarity_scores(text_result)['compound']>0:
#         emotion = 'Positive'
#     elif sia.polarity_scores(text_result)['compound']<0:
#         emotion = 'Negative'
#     else :
#         emotion = 'Neutral '
#     return emotion

In [ ]:
def get_language(audio,duration=30):
    clip_audio = whisper.pad_or_trim(audio,length=SAMPLE_RATE*duration)
    result = wisper_model.transcribe(clip_audio)
    return result['language']

def speech2text(audio_file,meta_info=[]):
    audio = whisper.load_audio(audio_file)
    audio_language = get_language(audio)
    if meta_info is None:
        meta_info = ['','']
    if audio_language =='en':
        result = wisper_model.transcribe(audio)
    else:
        print("Spoken Language is not english.  Translating {} to english".format(audio_language))
        # result = wisper_model.transcribe(audio, task='translate')
        return 
    
    result = wisper_model.transcribe(audio_file)

    return {
            'filename':os.path.basename(audio_file),
            'content':{
                'text':result['text'],
                'language':audio_language,
                'Customer':meta_info[0],
                'Assigned': meta_info[1],
                'properties':{
                    'segments':len(result['segments']),
                    'audio_duration':audio.shape[0]/SAMPLE_RATE,
                    }
                }
            }

In [ ]:
from tqdm import tqdm
conversation_dict['Data'] = list()
for fname in tqdm(audio_files):
    file_path = os.path.join(data_path,fname)
    meta_information = excel_data[['Opportunity','Assigned']][excel_data['Ameyo Recording URL'].str.contains(fname)].values.tolist()[0]
    conversation_dict['Data'].append(speech2text(audio_file=file_path,meta_info=meta_information))
    break

# with open("../data/Audio_data.json", "w") as outfile:
#     json.dump(conversation_dict, outfile,indent=4)

In [ ]:
with open("../data/Audio_data.json", "w") as outfile:
    json.dump(conversation_dict, outfile,indent=4)

### Asking Chat gpt to make some conversations 

#### Load the Conversations as Json Loader 

In [ ]:

# with open("../data/Audio_data.json" ,'r') as fd:
#     data = json.load(fd)

# for content in data['Data']:
#     content['content']['text'] = "{} is assigned to customer {}. Conversation:{}".format(content['content']['Assigned'],content['content']['Customer'],content['content']['text'])

# with open("../data/Audio_data_modified.json", "w") as outfile:
#     json.dump(data, outfile,indent=4)

In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["language"] = record.get("language")
    metadata["audio_properties"] = record.get("properties")
    metadata["Customer"] = record.get("Customer")
    metadata["Assigned"] = record.get("Assigned")
    return metadata

loader = JSONLoader(
    file_path='../data/Audio_data_modified.json',
    jq_schema='.Data[].content',
    content_key="text",
    metadata_func=metadata_func
)

In [ ]:
conversation = loader.load()
conversation[:4]

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500,chunk_overlap=0) # hyperparameters 
texts = text_splitter.split_documents(documents=conversation)  # metadatas=[doc.metadata for doc in docs]


In [ ]:
print("Total texts documents ",len(texts))
print(type(texts),type(texts[0]))

### Conversation data into Embedddings 

In [ ]:
from langchain.embeddings.base import Embeddings
from typing import List
from sentence_transformers import SentenceTransformer
from langchain.embeddings.openai import OpenAIEmbeddings

import openai
import os 
os.environ['OPENAI_API_KEY'] = "" 

class LocalHuggingFaceEmbeddings(Embeddings):
    def __init__(self, model_id): 
        # Should use the GPU by default
        self.model = SentenceTransformer(model_id)
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents using a locally running
           Hugging Face Sentence Transformer model
        Args:
            texts: The list of texts to embed.
        Returns:
            List of embeddings, one for each text.
        """
        embeddings =self.model.encode(texts)
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        """Embed a query using a locally running HF 
        Sentence trnsformer. 
        Args:
            text: The text to embed.
        Returns:
            Embeddings for the text.
        """
        embedding = self.model.encode(text)
        return list(map(float, embedding))

local_embeddings = LocalHuggingFaceEmbeddings('multi-qa-mpnet-base-dot-v1')
open_ai_embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS
vectorstore =  FAISS.from_documents(documents=texts,
                                    embedding = open_ai_embeddings
                                    ) # turn dcos into Vectors and store them in RAM also add metadata 
vectorstore.save_local('../data/faiss_dmac_conv')

## Retrival QA
Better version of Query 

###  underneath the hood 
- Underneath the hood is taking our query that we're going which is send to embed.
- Its turned into a vector, send it into device vector store.
- Then it's going to find similar vectors to that vector and it's going to bring those vectors back.
- Translate them into the text and we're going to send this as the context exactly like the vector chain, but with a bit of different parameters, but nothing very different here.

In [ ]:
from langchain.chains import RetrievalQA ,VectorDBQAWithSourcesChain ,RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

new_vector_store = FAISS.load_local("../data/faiss_dmac_conv/",embeddings=open_ai_embeddings)

### QA Retrival Chain to Answer specific questions 

In [ ]:
prompt_template = """
Analyze conversations between customer and sales person. 
Use the context to answer.If question is related to customers , answer in points with names of the customers.
If you don't know the answer, just say No idea.

{context}

Question: {question}
Answer in Points: 
"""

prompt = PromptTemplate(input_variables=["context","question"],template=prompt_template)

In [ ]:
chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(
                        llm=OpenAI(temperature=0),
                        chain_type="stuff",
                        retriever = new_vector_store.as_retriever(),
                        chain_type_kwargs=chain_type_kwargs
                        )

In [ ]:
query = "What product Faraht Khan is interested in ?"
res = qa.run(query)
print("query:{}".format(query))
print("response:\n{}".format(res))


In [ ]:
query = "What is next action with Faraht Khan ?"
res = qa.run(query)
print("query:{}".format(query))
print("response:\n{}".format(res))

In [ ]:
query = "Customers whos' unit price is more than 1.6 million "
res = qa.run(query)
print("query:{}".format(query))
print("response:\n{}".format(res))


In [ ]:
query = "Summarize conversations with Farhat Khan"
res = qa.run(query)
print("query:{}".format(query))
print("response:\n{}".format(res))

In [ ]:
query = "For Faraht khan any follow up leads"
res = qa.run(query)
print("query:{}".format(query))
print("response:\n{}".format(res))
